In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
import pickle
%matplotlib inline

In [4]:
dataset = os.path.join(parentdir,'book_data\BX_sample_item2159_user841')     
ratings = pd.read_csv(dataset, sep=',',
                      names=['user', 'item', 'rating', 'gender'], header=0)

data = os.path.join(parentdir,'book_data/book_author_gender.csv')

items = pd.read_csv(data, sep=',',
                      names=['item','gender'], header=0)

In [5]:
#ratings.drop("index", axis=1)
ratings = ratings.reset_index(drop=True)


In [6]:
algo_uu = uknn.UserUser(20)
algo_ii = iknn.ItemItem(20)
algo_als = als.BiasedMF(50)
algo_funkSVD = funksvd.FunkSVD(50)

In [7]:
items_n = ratings.item.nunique()
items_n

2159

In [8]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    
    # now we run the recommender
    recs = batch.recommend(fittable, users, items_n)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [9]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs.append(eval('ALS', algo_als, train, test))
    all_recs.append(eval('FunkSVD', algo_funkSVD, train, test))

C:\Users\josse\Git-kode projects\Speciale\Fair-Recommendations\lenskit\algorithms\user_knn.py:86: RuntimeWarning: invalid value encountered in true_divide
  uir.values[sp:ep] /= n
C:\Users\josse\Git-kode projects\Speciale\Fair-Recommendations\lenskit\algorithms\user_knn.py:148: RuntimeWarning: invalid value encountered in greater_equal
  mask = np.abs(i_sims >= 1.0e-10)
C:\Users\josse\Git-kode projects\Speciale\Fair-Recommendations\lenskit\algorithms\user_knn.py:154: RuntimeWarning: invalid value encountered in greater_equal
  mask = np.logical_and(mask, i_sims >= self.min_sim)
C:\Users\josse\Git-kode projects\Speciale\Fair-Recommendations\lenskit\algorithms\user_knn.py:86: RuntimeWarning: invalid value encountered in true_divide
  uir.values[sp:ep] /= n
C:\Users\josse\Git-kode projects\Speciale\Fair-Recommendations\lenskit\algorithms\user_knn.py:148: RuntimeWarning: invalid value encountered in greater_equal
  mask = np.abs(i_sims >= 1.0e-10)
C:\Users\josse\Git-kode projects\Speciale\

In [10]:
test_data = pd.concat(test_data, ignore_index=True)
all_recs = pd.concat(all_recs, ignore_index=True)

In [11]:
all_recs_joined =  pd.merge(all_recs, items.set_index('item'), on='item', how="left")
print(all_recs_joined.head(20))
dummy = pd.get_dummies(all_recs_joined["gender"])
dummy.head()
all_recs_joined["female"] = dummy.female
all_recs_joined["male"] = dummy.male

all_recs_joined.head(20)


         item      score  user  rank Algorithm  gender
0   106436329  12.366923   638     1  ItemItem    male
1   110099531  12.348494   638     2  ItemItem    male
2   100665513  12.086551   638     3  ItemItem    male
3   103026018  11.930233   638     4  ItemItem    male
4   114116734  11.741008   638     5  ItemItem    male
5   107191337  11.729906   638     6  ItemItem    male
6   104602439  11.703857   638     7  ItemItem  female
7   103192436  11.671852   638     8  ItemItem    male
8   101888343  11.493164   638     9  ItemItem  female
9   116034509  11.471156   638    10  ItemItem    male
10  103368220  11.417829   638    11  ItemItem    male
11  102223499  11.267139   638    12  ItemItem    male
12  115104183  11.223063   638    13  ItemItem    male
13  110582208  11.157895   638    14  ItemItem    male
14  108277189  11.155243   638    15  ItemItem  female
15  101575904  11.122741   638    16  ItemItem    male
16  102603458  11.077391   638    17  ItemItem    male
17  100609

,item,score,user,rank,Algorithm,gender,female,male
0,106436329,12.366923,638,1,ItemItem,male,0,1
1,110099531,12.348494,638,2,ItemItem,male,0,1
2,100665513,12.086551,638,3,ItemItem,male,0,1
3,103026018,11.930233,638,4,ItemItem,male,0,1
4,114116734,11.741008,638,5,ItemItem,male,0,1
5,107191337,11.729906,638,6,ItemItem,male,0,1
6,104602439,11.703857,638,7,ItemItem,female,1,0
7,103192436,11.671852,638,8,ItemItem,male,0,1
8,101888343,11.493164,638,9,ItemItem,female,1,0
9,116034509,11.471156,638,10,ItemItem,male,0,1


In [12]:
test_data.to_pickle("test_data_BX")
all_recs_joined.to_pickle("all_recs_joined_BX")

In [13]:
def evaluate_genre(genre): 
    
    providers = ["gender"]
    proItems_genre_n =  all_recs_joined.loc[ all_recs_joined[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    rla.add_metric("rKL")
    rla.add_metric("rRD")
    #rla.add_metric("APCR")
    #rla.add_metric("nd_APCR")
    rla.add_metric("equal_ex")
    rla.add_metric("ndcg")
    results= rla.compute(all_recs_joined, test_data, genre, providers)
    #results_horror.groupby('Algorithm').mean()
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [ ]:
testproviders = ["female"]
#testproviders = list(all_recs_joined.iloc[:,8:])
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)    

female


C:\Users\josse\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


8.403958142413677
13.91296148585999
15.288898072679581
8.392757195225846
14.158714651555329
15.288898072679581
8.186396913006888
13.559221694874552
15.246734775924955
8.29055758695083
13.749998676743383
15.277438743766828
8.496182005908304
14.04382762689995
15.10226530953744
8.502598194433334
14.074210560707815
15.10226530953744
7.982095867035338
13.136930156052623
14.687478251578884
8.193897708555252
13.7254154944433
14.655479748375171
8.471461145121683
14.006560539981797
15.250323071386807
8.438794059172873
13.946251378825952
15.250323071386807
7.730100058252043
12.794010093330595
14.033341733594863
8.111627083401066
13.321145230915667
14.647788031900642
8.457690529037468
13.872986896980706
15.172327777509764
8.476339109583503
14.047358175454622
15.172327777509764
7.816339973666355
12.869831108302426
14.406614354398751
7.7631706487917125
12.883409884226904
14.431241216510447
8.46182473217325
14.113553188582
15.170995374166983
8.465348851307557
14.113379613636784
15.170995374166983
8.

8.448302326296558
13.835343530735319
15.22702965493649
8.417399597539774
14.159285706228008
15.22702965493649
7.893257373738474
13.073088480590798
14.883126324041509
7.672708513171239
12.75336790301706
14.580263778835022
8.42945575239587
14.164251446840433
14.939876021492177
8.43086434885887
14.071953628684374
14.939876021492177
8.187589412632942
13.814251004309265
14.900463788434285
7.991883627439316
12.957328152872032
14.687478251578884
8.46341062519281
13.888075960033763
15.296259731291789
8.454959709453643
13.94638302039166
15.296259731291789
7.928148303928209
13.410034167841179
14.003723297707143
7.8387476399000136
13.251021288154025
13.695540519414402
8.457780063275418
14.070845942212603
15.33538618038373
8.398858804630509
14.399044381099092
15.33538618038373
8.097991875341616
13.569676044464591
15.07709686041071
8.095111769174713
13.361684341530005
14.808862146608186
8.441906099379485
14.011896843614744
15.301594046486718
8.438919069973885
14.132292831806442
15.301594046486718
7

8.384424305937312
13.9225224177754
15.205280025298268
8.411990342011057
13.800336468366874
15.205280025298268
8.129395971981902
13.544602111975811
15.050301235239349
7.858841731541186
12.772656450427288
14.585350870275438
8.437992890002066
14.151611472999894
15.176188310739368
8.513599061956477
14.046222819348838
15.176188310739368
8.16037849735319
13.656275340373114
14.763909070245393
7.901933503093663
12.84728204325298
14.854386902802684
8.337346788412917
13.480576208218213
15.210195242853379
8.384718380009902
13.707879842193844
15.210195242853379
8.120365927175694
13.374028824327365
15.243650410259173
7.983056580919491
13.322766099847929
15.13941997465852
8.45497980703821
14.037728992495783
15.276696506745818
8.473542714920956
13.907633493324344
15.276696506745818
8.026449533066705
13.26553564583036
14.882410354519639
7.69053901160148
12.933049736928032
14.640849849909225
8.409964677364608
13.966057376719721
15.16614083310923
8.45298834572083
13.824378188989368
15.16614083310923
8.1

...

15.16991486757372
8.440016904844558
14.161856678114026
15.16991486757372
8.080659871429981
13.640243638363007
14.888125242261628
8.272717831235742
13.724673337758395
14.94920270789151
8.38075790108566
13.985470094014726
14.857208710498417
8.39497903164916
13.902379186692368
14.857208710498417
8.008535948131033
13.428867519547296
14.72316495301094
7.992337434712266
13.015767685305615
14.63466990392072
8.479446128743692
13.89755166230079
15.22177301327255
8.422762929414569
14.194452828161804
15.22177301327255
8.175761291419866
13.52425969059638
15.069251003864618
8.238995550535257
13.658416773570806
15.217996414242284
8.406064774010996
14.180920123425318
15.324755434674993
8.456166826467115
14.013970676905979
15.324755434674993
8.152453876031988
13.496799940822944
14.961509456294488
7.884576839981003
12.868950424852873
14.897583990762831
8.249690086181564
13.668365446141394
15.355717857219526
8.239443430361325
13.320918739236442
15.355717857219526
8.072517536024204
13.241129907313468
15.

15.225597179390594
8.458256049555676
13.972920607943317
15.225597179390594
7.99157606273649
13.282379151483747
14.083322175427492
8.014545853936125
13.294973362312808
13.936990570758676
8.462196224050286
13.950552727633497
15.176188310739368
8.461692029398462
13.987306331315718
15.176188310739368
8.07472370558759
13.47161522648623
14.52748368728918
7.458534126479836
12.339931573790837
13.858284713696886
8.402322796635731
13.885171531792741
15.31756747617221
8.399955438612876
13.898619904230458
15.31756747617221
7.868040836125661
12.879348492721391
13.884842909029633
7.915475277349955
13.550008802667374
13.550280117960929
8.437545498801795
13.991960505018302
15.251841746282826
8.4058198764835
13.8731182350421
15.251841746282826
8.004251823022445
13.385759829965275
14.551826682271649
8.110529300555584
13.394500977652257
14.799918805859992
8.460123057110128
13.93613835502737
15.056688352819256
8.426891151399733
14.07303083075735
15.056688352819256
8.085841349041301
13.258395138504582
14.9

15.132015147532991
8.171092770878763
13.323208113553594
14.938715667571255
8.132087368829524
13.336706582675827
14.502604906019027
8.487437270590624
13.984811276762064
15.09440986604758
8.432414183846474
14.017864188696851
15.09440986604758
8.190019089413376
13.564365615400117
14.836053938749092
8.183818375187448
13.884236655533892
14.681626344829846
8.479101124770565
14.145159311391222
15.225597179390594
8.458498618233511
14.293114792550682
15.225597179390594
8.240238752165371
13.479736714439161
15.117677582475348
8.138216895807442
13.504326681400201
14.830363521523111
8.497401088444168
14.326820691730793
14.908389087961499
8.44038713988203
14.043225534024995
14.908389087961499
8.145441922833166
13.366094058318222
14.650911067788098
7.952869883311062
13.046022159239135
14.557126354755217
8.460039001257181
14.083759179093388
15.156681553431497
8.464183758294862
13.757962114789304
15.156681553431497
8.078714484608776
13.401824589247576
14.890051381267813
8.105712473938725
13.49675127592

13.970462211513569
15.113889343913126
8.103266348861585
13.446719085891909
14.907941016174993
8.162539255223145
13.400025884419927
14.934676278904394
8.43329251463914
13.872912413917518
15.356828960463753
8.423382443931837
14.058629572368158
15.356828960463753
8.076851650200869
13.420971943733786
14.78755447422731
8.161352896849811
13.388349350523105
14.646395242379423
8.480885194611915
14.022012152707106
15.103605405675268
8.462275826650957
13.99460707628958
15.103605405675268
7.955699267748811
12.95979029605301
14.826294613961462
8.077656005834928
13.333970905480973
14.735390487409992
8.489807004864128
13.789479149322862
15.098392834011443
8.453282012740386
13.995879330542234
15.098392834011443
8.133393709196369
13.427072179612173
14.951080724592279
7.86094426604204
12.741088892286676
14.71313503362658
8.433881209660601
13.85726750613542
15.374590344575006
8.446775242174766
14.101066914610604
15.374590344575006
8.07012216388062
13.615283765818745
14.688589451848987
8.173837097852964


13.315645790194818
14.911392520333445
8.184498392084961
13.331213282125898
15.001647810270807
8.478458664121412
13.98695663988045
15.077708193393473
8.531179416813693
14.098208951917474
15.077708193393473
7.848885933423039
13.025158003758122
14.756870983734862
7.804047336695676
12.79926035343662
14.845727869139974
8.499314042633596
13.937724623838736
15.117822407151488
8.44437166933331
14.085805079205567
15.117822407151488
8.08722064724105
13.198713614909133
14.683142066756627
7.404519015408998
12.281189208735226
14.175104581638488
8.44468926934672
14.04458463672977
15.103605405675268
8.467592338669578
14.09426514429554
15.103605405675268
7.847021482993396
12.81424272328473
14.877597827791579
7.868519314884587
12.865833164118175
14.788074247065902
8.43569045049034
13.659471109984835
14.895957378088719
8.454202213063713
13.666117458032776
14.895957378088719
8.15161860984618
13.416966164586672
14.743063406276914
8.282257542291442
14.080403927959654
14.630917338675488
8.429830788705164
14

In [ ]:
output = pd.concat(allResults, keys=testproviders)
output

In [ ]:
output.to_csv('results_08.05_BX_sample_item2159_user841_cut10_log2')